In [ ]:
library(randomForest)
library(caret)
library(ggplot2)
library(reshape2)  
set.seed(123)

simulate_data <- function(n) {
  data <- data.frame(
    chemical_id = 1:n,                     
    order = sample(0:2, n, replace = TRUE),  
    A0 = runif(n, 0.1, 2),                
    k = runif(n, 0.01, 1),              
    t = runif(n, 0, 100),                   
    temperature = runif(n, 273, 373),        
    activation_energy = runif(n, 50, 150)   
  )
  
  
  data$concentration <- apply(data, 1, function(row) {
    order <- as.numeric(row["order"])
    A0 <- as.numeric(row["A0"])
    k <- as.numeric(row["k"])
    t <- as.numeric(row["t"])
    
    if (order == 0) {
     
      A <- A0 - k * t
    } else if (order == 1) {
      
      A <- A0 * exp(-k * t)
    } else if (order == 2) {
     
      A <- 1 / (1 / A0 + k * t)
    }
    
   
    if (A < 0) A <- 0
    return(A)
  })
  
  return(data)
}

data <- simulate_data(200)

head(data)

cor_matrix <- cor(data[, c("A0", "k", "t", "temperature", "activation_energy", "concentration")])

melted_cor_matrix <- melt(cor_matrix)

ggplot(data = melted_cor_matrix, aes(x = Var1, y = Var2, fill = value)) +
  geom_tile() +
  geom_text(aes(label = round(value, 2)), color = "black") +  # Add correlation numbers
  scale_fill_gradient2(low = "blue", high = "red", mid = "white", midpoint = 0,
                       limit = c(-1, 1), space = "Lab", name = "Correlation") +
  theme_minimal() +
  labs(title = "Correlation Matrix of Key Variables") +
  theme(axis.text.x = element_text(angle = 45, vjust = 1, size = 12, hjust = 1))

data$log_A0 <- log(data$A0)
data$log_k <- log(data$k)

trainIndex <- createDataPartition(data$concentration, p = 0.8, list = FALSE)
trainData <- data[trainIndex, ]
testData <- data[-trainIndex, ]

model <- randomForest(concentration ~ order +
                        A0 + k + t + temperature + activation_energy, data = trainData)
predictions <- predict(model, testData)

rmse <- sqrt(mean((testData$concentration - predictions)^2))
cat("RMSE of the model:", rmse, "\n")

r_squared <- 1 - (sum((testData$concentration - predictions)^2)
                  / sum((testData$concentration - mean(testData$concentration))^2))
cat("R-squared of the model:", r_squared, "\n")

importance(model)
varImpPlot(model)

# 9. Cross-validation example (10-fold cross-validation)
control <- trainControl(method = "cv", number = 10)
model_cv <- train(concentration ~ order + A0 + k + t + temperature + activation_energy, data = trainData, method = "rf", trControl = control)
print(model_cv)